In [1]:
import numpy as np
import theano
import theano.tensor as T

import lasagne
import lasagne.layers.dnn 

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 5105)


In [222]:
input = np.random.randn(300, 3, 20, 20, 20).astype(np.float32)
target = ((np.random.randn(300)+1)/2).astype(np.int32)                                     

In [258]:
target

array([ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  1,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  2,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  1,  0,  0,  0

In [223]:
dtensor5 = T.TensorType('float32', (False,)*5)
input_var = dtensor5('input')
target_var = T.ivector('target')

network = lasagne.layers.InputLayer((300, 3, 20, 20, 20), input_var=input_var)
network = lasagne.layers.dnn.Conv3DDNNLayer(lasagne.layers.DropoutLayer(network, 0.5), 
                                            num_filters=10, filter_size=(4,4,4))
network = lasagne.layers.dnn.Conv3DDNNLayer(lasagne.layers.DropoutLayer(network, 0.5), 
                                            num_filters=10, filter_size=(4,4,4))
network = lasagne.layers.DenseLayer(network, num_units=30, nonlinearity=lasagne.nonlinearities.softmax)
network = lasagne.layers.DenseLayer(network, num_units=2, nonlinearity=lasagne.nonlinearities.softmax)

prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var).mean()

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)

train_fn = theano.function([input_var, target_var], loss, updates=updates)

In [255]:
train_fn(input, target)

array(0.3995228707790375, dtype=float32)

In [256]:
test_fn = theano.function([input_var], prediction)

In [257]:
test_fn(input)

array([[ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543444],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,  0.15543449],
       [ 0.84456551,  0.15543446],
       [ 0.84456551,

In [21]:
np.random.randn(1, 1, 20, 20, 20).

In [57]:
print(prediction)

Softmax.0


In [113]:
a = T.dscalar('a')
b = T.dscalar('b')
z = a ** a - b ** b
fun = theano.function([a, b], T.grad(z, [a, b]))
fun(1, 1)

[array(1.0), array(-1.0)]

In [141]:
v = T.dvector('v')
z = v ** 2
w, updates = theano.scan(lambda i: T.grad(z[i], v), sequences=T.arange(v.shape[0]))
fun = theano.function([v], w)
fun([1, 1, 2,3, 6,45, 8,6,67])

array([[   2.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    2.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    4.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    6.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,   12.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,   90.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,   16.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,   12.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  134.]])